In [7]:
import sys
sys.path.insert(1, '..')
import tools.read_data as rd
import pandas as pd
import unittest
import time
import json
import importlib
for k,v in list(sys.modules.items()):
    if k.startswith('tools'):
        importlib.reload(v)

## Read in the data

In [8]:
# read in the json data using a generator method
patient_json_list = []
start = time.time()
for json_obj in rd.read_json_files('data'):
    patient_json_list.append(json_obj)
end = time.time()
print(end - start)

Error: Could not read file - please pass a different directory or filename
1.468338966369629


In [ ]:
# read in the json data using a standard method
patient_json_list_alt = []
start = time.time()
pfl = rd.get_patient_file_list('data')
for json_obj in pfl:
    patient_json_list_alt.append(rd.read_patient_file('data', json_obj))
end = time.time()
print(end - start)

3.676032781600952


In [ ]:
# we have also implemented functions to read in the data from a database or an API - these are not implemented in this example
# patient_json_list = rd.get_json_objects_from_API('https://www.example-patient-api.com/get-patient-FHIR-data')

In [ ]:
# check the two different methods correctly calculated the same list
assert patient_json_list == patient_json_list_alt

## Run tests for data quality / correctness

In [ ]:
# run all tests on the incoming data
from tools.data_tests import TestFHIRData

test_runner = unittest.TextTestRunner()
for json_obj in patient_json_list:
    TestFHIRData.JSON_OBJ = json_obj
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestFHIRData)
    test_results = test_runner.run(test_suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.346s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.262s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.150s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.485s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 11.012s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.420s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.327s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.216s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.494s

OK
....
----------------------------------------------------------------------
Ran 4

In [ ]:
# example test failure
with open("resources/bad_example.json") as f:
    bad_json_obj = json.load(f)

TestFHIRData.JSON_OBJ = bad_json_obj
test_suite = unittest.TestLoader().loadTestsFromTestCase(TestFHIRData)
test_results = test_runner.run(test_suite)

FFFF
FAIL: test_all_fields_in_patient (tools.data_tests.TestFHIRData)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/joshuastapleton/Desktop/EMIS_interview/exa-data-eng-assessment/tools/data_tests.py", line 43, in test_all_fields_in_patient
    self.assertIn(field, expected_fields, msg=f"{field} field not found in expected fields list")
AssertionError: 'subject' not found in ['resourceType', 'fhir_comments', 'id', 'implicitRules', 'implicitRules__ext', 'language', 'language__ext', 'meta', 'contained', 'extension', 'modifierExtension', 'text', 'active', 'active__ext', 'address', 'birthDate', 'birthDate__ext', 'communication', 'contact', 'deceasedBoolean', 'deceasedBoolean__ext', 'deceasedDateTime', 'deceasedDateTime__ext', 'gender', 'gender__ext', 'generalPractitioner', 'identifier', 'link', 'managingOrganization', 'maritalStatus', 'multipleBirthBoolean', 'multipleBirthBoolean__ext', 'multipleBirthInteger', 'multi

## Basic / local implementation of pipeline using filesystem

In [ ]:
from fhir.resources.patient import Patient
from fhir.resources.bundle import Bundle

FHIR_patient_object_list = [Patient.parse_obj(Bundle.parse_obj(patient_json).entry[0].resource) for patient_json in patient_json_list]
patient_df = rd.patients_to_dataframe(FHIR_patient_object_list).drop(columns=['resource_type']) # we can drop this column because it is constant by definition

In [ ]:
# write the raw tabular data to a csv file. This needs to be normalized and cleaned before it can be used for analysis.
patient_df.to_csv('data_output/patient_data_tabular_raw.csv', index=False)

In [ ]:
# 1NF normalization - each table cell should have a single value
# the columns in the dataframe in need of normalization are extension, address, maritalStatus, name, telecom, etc.
# a naive solution would be to explode the columns that are lists. This, however, tends to become monolithic, as the number of table rows grows exponentially.
print("exploding column: extension")
patient_exploded_df = patient_df.explode('extension') # start by exploding extension - the first column of type list
for column in patient_df.columns.drop('extension'):
    if type(patient_df[column][0]) == list:
        print("exploding column: " + column)
        patient_exploded_df = patient_exploded_df.explode(column)

patient_exploded_df.to_csv('data_output/1NF_data/patient_data_tabular.csv', index=False)

exploding column: address
exploding column: communication
exploding column: identifier
exploding column: name
exploding column: telecom


In [ ]:
# 2NF normalization - create additional tables for initial table cells with multiple/list entires
# this is a more complex solution, but it is more scalable, easier to maintain, and there is less data redundancy
patient_df_2NF = patient_df.copy()

for column in patient_df_2NF.columns:
    if type(patient_df_2NF[column][0]) == list:
        print("exploding column: " + column)
        patient_exploded_df = patient_df_2NF.explode(column)
        patient_df_2NF = patient_df_2NF.drop(columns=[column])

        # drop all columns from the exploded dataframe that are in the original dataframe except ID
        NF_columns = list(patient_df_2NF.columns)
        NF_columns.remove('id')
        patient_exploded_df.drop(columns=NF_columns, inplace=True)
        patient_exploded_df.to_csv('data_output/2NF_data/patient_data_tabular_' + column + '.csv', index=False)

# finally, write the original table with all multi-value columns removed to a csv file
patient_df_2NF.to_csv('data_output/2NF_data/patient_data_tabular.csv', index=False)

exploding column: extension
exploding column: address
exploding column: communication
exploding column: identifier
exploding column: name
exploding column: telecom


In [ ]:
# #  we can further expand the table by identifying values which are FHIR objects and splitting them up by field, however this can get tricky with string parsing
# patient_1NF_df = pd.read_csv('data_output/1NF_data/patient_data_tabular.csv')
# for column in patient_1NF_df.columns:
#     # if the column starts with 'resource'
#     first_column_value = patient_1NF_df[column].values[0]
#     if type(first_column_value) == str and first_column_value.startswith('resource_type'):
#         print("Fields of column:", column)
#         for field in first_column_value.split(' '):
#             print("--",field)

## SQL implementation of pipeline with database

### First, create the tables

In [1]:
import sys
sys.path.insert(1, '..')
import pandas as pd
import tools.create_database as cd
import tools.update_database as ud
from sqlalchemy import text
import importlib
for k,v in list(sys.modules.items()):
    if k.startswith('tools'):
        importlib.reload(v)

In [2]:
# get/open the connection to the patient database, dropping contents if it already exists
patient_database_conn = cd.connect_to_sqla_server()
patient_database_conn.execute(text("DROP TABLE IF EXISTS patient"))
patient_database_conn.execute(text("DROP TABLE IF EXISTS address"))
patient_database_conn.execute(text("DROP TABLE IF EXISTS communication"))
patient_database_conn.execute(text("DROP TABLE IF EXISTS extension"))
patient_database_conn.execute(text("DROP TABLE IF EXISTS identifier"))
patient_database_conn.execute(text("DROP TABLE IF EXISTS name"))
patient_database_conn.execute(text("DROP TABLE IF EXISTS telecom"))

In [3]:
# create the tables in the patient database with 2NF standards - these correspond to the csv files in the data_output 2NF folder generated in the previous steps
TWONF_CREATE_PATIENT_TABLE_SQL = """
        CREATE TABLE patient (
            fhir_comments VARCHAR(255),
            id VARCHAR(36),
            implicitRules VARCHAR(255),
            implicitRules__ext VARCHAR(255),
            language VARCHAR(255),
            language__ext VARCHAR(255),
            meta VARCHAR(255),
            contained VARCHAR(255),
            modifierExtension VARCHAR(255),
            text VARCHAR(255),
            active BOOLEAN,
            active__ext VARCHAR(255),
            birthDate DATE,
            birthDate__ext VARCHAR(255),
            contact VARCHAR(255),
            deceasedBoolean BOOLEAN,
            deceasedBoolean__ext VARCHAR(255),
            deceasedDateTime DATE,
            deceasedDateTime__ext VARCHAR(255),
            gender VARCHAR(255),
            gender__ext VARCHAR(255),
            generalPractitioner VARCHAR(255),
            link VARCHAR(255),
            managingOrganization VARCHAR(255),
            maritalStatus VARCHAR(255),
            multipleBirthBoolean BOOLEAN,
            multipleBirthBoolean__ext bool,
            multipleBirthInteger int,
            multipleBirthInteger__ext VARCHAR(255),
            photo string
        );
    """

# ud.create_table(TWONF_CREATE_PATIENT_TABLE_SQL, patient_database_conn)

# create the address table
TWONF_CREATE_ADDRESS_TABLE_SQL = """
    CREATE TABLE address (
        id bool,
        address string
    );
"""
ud.execute_sql(TWONF_CREATE_ADDRESS_TABLE_SQL, patient_database_conn)

# create the communication table
TWONF_CREATE_COMMUNICATION_TABLE_SQL = """
    CREATE TABLE communication (
        id string,
        communication string
    );
"""
ud.execute_sql(TWONF_CREATE_COMMUNICATION_TABLE_SQL, patient_database_conn)

# create the extension table
TWONF_CREATE_EXTENSION_TABLE_SQL = """
    CREATE TABLE extension (
        id string,
        extension string
    );
"""
ud.execute_sql(TWONF_CREATE_EXTENSION_TABLE_SQL, patient_database_conn)

# create the identifier table
TWONF_CREATE_IDENTIFIER_TABLE_SQL = """
    CREATE TABLE identifier (
        id string,
        identifier string
    );
"""
ud.execute_sql(TWONF_CREATE_IDENTIFIER_TABLE_SQL, patient_database_conn)

# create the name table
TWONF_CREATE_NAME_TABLE_SQL = """
    CREATE TABLE name (
        id string,
        name string
    );
"""
ud.execute_sql(TWONF_CREATE_NAME_TABLE_SQL, patient_database_conn)

# create the telecom table
TWONF_CREATE_TELECOM_TABLE_SQL = """
    CREATE TABLE telecom (
        id string, 
        telecom string
    );
"""
ud.execute_sql(TWONF_CREATE_TELECOM_TABLE_SQL, patient_database_conn)

True

### Second, populate the tables

In [4]:
telecom_data = pd.read_csv('data_output/2NF_data/patient_data_tabular_telecom.csv')

for index, row in telecom_data.iterrows():
    patient_id = "\""+row['id']+"\""
    # enclose with double quotes
    address = "\""+row['telecom']+"\""
    insert_telecom_sql = "INSERT INTO telecom (id, telecom) VALUES ({}, {})".format(patient_id, address)
    patient_database_conn.execute(text(insert_telecom_sql))

### Example queries

In [5]:
result = patient_database_conn.execute(text("SELECT * FROM telecom"))
df = pd.DataFrame(result.all(), columns=result.keys())
df

,id,telecom
0,8a3247d3-a54c-43f2-2c5d-a8f5e28ff588,resource_type='ContactPoint' fhir_comments=Non...
1,b0f40536-9dc8-2ea0-0bbf-467a69f5e3ad,resource_type='ContactPoint' fhir_comments=Non...
2,09e292d4-f186-331c-ed95-c503acabc54e,resource_type='ContactPoint' fhir_comments=Non...
3,10bf6da8-ffa1-6913-a119-726634be754c,resource_type='ContactPoint' fhir_comments=Non...
4,0d55a582-07fe-a897-776c-3ab5e48cd457,resource_type='ContactPoint' fhir_comments=Non...
...,...,...
74,f406a4e8-821b-0c9a-c8ec-09ad0f1fe9c6,resource_type='ContactPoint' fhir_comments=Non...
75,aad9d04b-bb30-2f47-d5dd-888b3b7bd831,resource_type='ContactPoint' fhir_comments=Non...
76,aa4eae2c-733a-35f9-8869-d33a6015db23,resource_type='ContactPoint' fhir_comments=Non...
77,cf3ce382-cceb-1557-89ac-b751a9e0e65d,resource_type='ContactPoint' fhir_comments=Non...
